In [ ]:
def xs_to_shp(xs_csv,outname):
    '''
    This function converts a .csv of a surveyed cross section to a shapefile of measurements
    The .csv must be ordered from left to right (looking downstream)
    outname is the desired name for the output of the shapefile, must end in .shp
    '''
    # Read in csv, drop nan rows
    xs = pd.read_csv(xs_csv)
    xs = xs.dropna(how='all')
    
    # Create variable of endpoint coordinates
    coords = xs.dropna(subset=["x"])
    
    # Define left and right end points
    lep = coords.iloc[0]
    rep = coords.iloc[1]
    
    # Calculate angle of cross-section
    angle = math.atan((lep.y-rep.y)/(lep.x-rep.x))
    print(angle)
    
    # Calculate UTM easting and northing
    if lep.x <= rep.x and lep.y <= rep.y:
        xs['east'] = lep.x + (math.cos(angle)*(xs.dist-lep.dist))
        xs['nrth'] = lep.y + (math.sin(angle)*(xs.dist-lep.dist))
        print("west-east, south-north")
    elif lep.x <= rep.x and lep.y >= rep.y:
        xs['east'] = lep.x + (math.cos(angle)*(xs.dist-lep.dist))
        xs['nrth'] = lep.y + (math.sin(angle)*(xs.dist-lep.dist))
        print("west-east, north-south")
    elif lep.x >= rep.x and lep.y <= rep.y:
        xs['east'] = lep.x + (math.cos(angle)*(xs.dist-lep.dist))
        xs['nrth'] = lep.y + (math.sin(angle)*(xs.dist-lep.dist))
        print("east-west, south-north")
    elif lep.x >= rep.x and lep.y <= rep.y:
        xs['east'] = lep.x + (math.cos(angle)*(xs.dist-lep.dist))
        xs['nrth'] = lep.y - (math.sin(angle)*(xs.dist-lep.dist))
        print("east-west, north-south")
    
    # Combine east and nrth column to a shapely Point() object
    xs['geometry'] = xs.apply(lambda x: Point((float(x.east), float(x.nrth))), axis=1)
    
    # Create shapefile in UTM from dataframe
    xs = geopandas.GeoDataFrame(xs, geometry='geometry')
    xs.crs = {'init' :'epsg:32610'}
    xs.to_file(outname, driver='ESRI Shapefile')